In [1]:
from PIL import Image
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import cv2
import layoutparser as lp
from pdf2image import convert_from_path
from paddleocr import PaddleOCR, draw_ocr
import tensorflow as tf
import numpy as np
import pandas as pd
import os

In [232]:

def intersection(box_1, box_2):
    return [box_2[0], box_1[1],box_2[2], box_1[3]]

def iou(box_1, box_2):

    x_1 = max(box_1[0], box_2[0])
    y_1 = max(box_1[1], box_2[1])
    x_2 = min(box_1[2], box_2[2])
    y_2 = min(box_1[3], box_2[3])

    inter = abs(max((x_2 - x_1, 0)) * max((y_2 - y_1), 0))
    if inter == 0:
        return 0

    box_1_area = abs((box_1[2] - box_1[0]) * (box_1[3] - box_1[1]))
    box_2_area = abs((box_2[2] - box_2[0]) * (box_2[3] - box_2[1]))

    return inter / float(box_1_area + box_2_area - inter)

def image_to_csv(image_file):

    # os.chdir(image_file.rsplit("/",1)[0])
    # # image = cv2.imread("ext_im.jpg")

    # image = image[..., ::-1]

    # load model
    # model = lp.models.PaddleDetectionLayoutModel(config_path="lp://PubLayNet/ppyolov2_r50vd_dcn_365e_publaynet/config",
    #                             threshold=0.5,
    #                             label_map={0: "Text", 1: "Title", 2: "List", 3:"Table", 4:"Figure"},
    #                             enforce_cpu=False,
    #                             enable_mkldnn=True)#math kernel library
    # # detect
    # layout = model.detect(image)

    # x_1=0
    # y_1=0
    # x_2=0
    # y_2=0

    # try:

    #     tables = []
    #     for l in layout:
    #         #print(l)
    #         if (l.type == 'Table') and (int(l.block.y_2) - int(l.block.y_1) >200) :
    #             x_1 = int(l.block.x_1)
    #             print(l.block.x_1)
    #             y_1 = int(l.block.y_1)
    #             x_2 = int(l.block.x_2)
    #             y_2 = int(l.block.y_2)

    #         break

    #     mod_file = image_file[:-4] + 'crop2' +'.jpg'

    #     image = cv2.imread(image_file)

    #     cv2.imwrite(mod_file, image[y_1:y_2,x_1:x_2])
    try:
        ocr = PaddleOCR(lang='en')
        image_in_path = image_file
        image_out_path = image_file[:-4] + '_alt.jpg'
        image_cv = cv2.imread(image_in_path)
        image_height = image_cv.shape[0]
        image_width = image_cv.shape[1]
        image_cv = np.asarray(image_cv)
        image_cv = cv2.cvtColor(image_cv, cv2.COLOR_BGR2GRAY)

        # Apply some preprocessing to improve the OCR quality
        # You can adjust the parameters according to your image
        # For example, you can use thresholding, morphological operations, etc.
        image_cv = cv2.GaussianBlur(image_cv, (3, 3), 0)
        # ret, image_cv = cv2.threshold(image_cv, 145, 255, cv2.THRESH_BINARY)
        image_cv = cv2.adaptiveThreshold(image_cv, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 9)
        kernel = np.ones((1, 1), np.uint8)
        image_cv = cv2.erode(image_cv, kernel, iterations =3) 
        image_cv = cv2.dilate(image_cv, kernel, iterations =3)

        image_cv = cv2.morphologyEx(image_cv,cv2.MORPH_CLOSE, kernel)
        cv2.imwrite(image_out_path, image_cv)
        # image_cv = Image.fromarray(image_cv)
        output = ocr.ocr(image_out_path)[0]

        boxes = [line[0] for line in output]
        texts = [line[1][0] for line in output]
        probabilities = [line[1][1] for line in output]
        image_boxes = image_cv.copy()

        for box,text in zip(boxes,texts):
            cv2.rectangle(image_boxes, (int(box[0][0]),int(box[0][1])), (int(box[2][0]),int(box[2][1])),(0,0,255),1)
            cv2.putText(image_boxes, text,(int(box[0][0]),int(box[0][1])),cv2.FONT_HERSHEY_SIMPLEX,1,(222,0,0),1)

        im = image_cv.copy()
            
        horiz_boxes = []
        vert_boxes = []

        for box in boxes:
            x_h, x_v = 0,int(box[0][0])
            y_h, y_v = int(box[0][1]),0
            width_h,width_v = image_width, int(box[2][0]-box[0][0])
            height_h,height_v = int(box[2][1]-box[0][1]),image_height

            horiz_boxes.append([x_h,y_h,x_h+width_h,y_h+height_h])
            vert_boxes.append([x_v,y_v,x_v+width_v,y_v+height_v])

            cv2.rectangle(im,(x_h,y_h), (x_h+width_h,y_h+height_h),(0,0,255),1)
            cv2.rectangle(im,(x_v,y_v), (x_v+width_v,y_v+height_v),(0,255,0),1)
            
        horiz_out = tf.image.non_max_suppression(
            horiz_boxes,
            probabilities,
            max_output_size = 1000,
            iou_threshold=0.1,
            score_threshold=float('-inf'),
            name=None
        )
        horiz_lines = np.sort(np.array(horiz_out))
        print(horiz_lines)

        im_nms = image_cv.copy()

        for val in horiz_lines:
            cv2.rectangle(im_nms, (int(horiz_boxes[val][0]),int(horiz_boxes[val][1])), (int(horiz_boxes[val][2]),int(horiz_boxes[val][3])),(0,0,255),1)
            
        vert_out = tf.image.non_max_suppression(
            vert_boxes,
            probabilities,
            max_output_size = 1000,
            iou_threshold=0.1,
            score_threshold=float('-inf'),
            name=None
        )
        vert_lines = np.sort(np.array(vert_out))

        for val in vert_lines:
            cv2.rectangle(im_nms, (int(vert_boxes[val][0]),int(vert_boxes[val][1])), (int(vert_boxes[val][2]),int(vert_boxes[val][3])),(255,0,0),1)
            
        out_array = [["" for i in range(len(vert_lines))] for j in range(len(horiz_lines))]
        print(np.array(out_array).shape)
        print(out_array)

        unordered_boxes = []

        for i in vert_lines:
            print(vert_boxes[i])
            unordered_boxes.append(vert_boxes[i][0])
            
        ordered_boxes = np.argsort(unordered_boxes)
        print(ordered_boxes)


        for i in range(len(horiz_lines)):
            for j in range(len(vert_lines)):
                resultant = intersection(horiz_boxes[horiz_lines[i]], vert_boxes[vert_lines[ordered_boxes[j]]] )

                for b in range(len(boxes)):
                    the_box = [boxes[b][0][0],boxes[b][0][1],boxes[b][2][0],boxes[b][2][1]]
                    if(iou(resultant,the_box)>0.1):
                        out_array[i][j] = texts[b]
                    
        out_array=np.array(out_array)
        final_out = pd.DataFrame(out_array)  #.to_csv(image_file[:-4]+'.csv', index = False,header= False)
        return final_out
    
    except Exception as e:
        return f"Error: {str(e)}"

In [234]:
import xlsxwriter


# image_file = "D:/img_to_text/converted_images2/BRW40490F351961_0043205.jpg"
path = "D:/img_to_text/converted_images2"
image_list = [f for f in [file for (path,dirs,file) in os.walk(path)][0] if (f.endswith('.jpg')) and (f[-8:-4] != '_alt') and (f[-8:-4] != 'crop')]

try:
    for n in image_list:
            og = n[:-6]
            og_list = [i for i in image_list if i[:-6] == og]
            with pd.ExcelWriter(og+'.xlsx', engine='xlsxwriter') as writer:
                for i in og_list:
                    image_list.remove(i)
                    df = image_to_csv(i)
                    if type(df) == pd.DataFrame:
                        df.to_excel(writer,i[-10:-4], index = 'False')
                    else:
                        print(df)
except Exception as e:
        print(str(e))
    


# image_to_csv(image_file)

# except Exception as e:
#     print(e)

[2024/02/01 15:19:01] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='C:\\Users\\Richard/.paddleocr/whl\\cls\\ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_box_type='quad', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='C:\\Users\\Richard/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max', e2e_model_dir=None, e2e_pgnet_mode='fast', e2e_pgnet_score_

: 

In [54]:
df = pd.read_csv("D:/img_to_text/converted_images/2018 Annual Chem1..csv")
df = df.iloc[1:,1:]
df = df.drop_duplicates()
df = df.dropna(how = 'all', axis = 0)
df = df.dropna(how = 'all', axis = 1)
df = df.dropna(axis = 1, thresh = 0.1*df.shape[0])
df = df.dropna(thresh = 0.1*df.shape[1], axis = 0)
df

,0,1,2,3,4,5,6,7
1,NaN,NaN,11-Jul-2018 12:00 11-Jul-2018 12:37 11-Jul-201...,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,pm,pm,pm,am,am
3,NaN,Lab Number:,2015484.5,2015484.1,2015484.2,2015484.3,2015484.4,2015484.5
4,"Drinking water metals suite, totals, trace",NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Total Silver,g/m3,0.00011,<0.00011,0.00011,<0.00011,0.00011,0.00011
6,Total Sodium,g/m3,NaN,8.0,12.1,10.8,8.9,9.1
7,Total Tin,g/m3,<0.00053,<0.00053,0.00053,<0.00053,0.00053,<0.00053
8,Total Uranium,g/m3,0.000021,0.00025,0.000028,0.000021,0.000032,0.000039
9,Total Zinc,g/m3,NaN,0.136,0.0118,0.0023,0.0143,0.102
10,NaN,Sample Name:,1807027-5,1807026-9,1807026-10,1807027-1,1807027-2,1807027-5


In [152]:
import re

files_path = "D:/img_to_text/converted_images"
csv_files = [f for f in [files for path,dirs,files in os.walk(files_path)][0] if f.endswith("csv")] # obtain csv file names
# lab_list = ['ecan|environment canterbury|esr|hill lab|watercare|eurofins|hill laboratories|aqualabs|water testing|analytica laboratories ltd|analytical research lab ltd|aqualab|asurequality|broadspectrum|cawthron|central environmental labs|christchurch cc|citilab|crl energy|dargaville hospital|esr|eurofins|far north envirolab|fonterra|food & health environmental|goodman fielder, meadow fresh|hamilton cc|hill labs|industrial chemistry services|invercargill cc|clifton|linnaeus laboratory|mb century|medlab timaru|microaquatech|new zealand aluminium smelters|niwa|rotorua lc environmental lab.|sgs nz, minerals div. westport|silver fern farms - takapau|tatua co-operative dairy co|tauranga cc|waimakariri dc|wairoa dc|water testing|watercare|whangarei dc',n.lower())']

def euro_extract(df, variant):
    output = pd.DataFrame(columns = ['sample_date','supply_component_name', 'parameter_determinand','original_value','laboratory', 'report_number'])
    sample_date = ''
    supply_component_name = ''
    parameter_determinand = ''
    original_value = ''
    laboratory = 'Eurofins ELS Limited'
    
    try:
        if variant == 'horizontal':
            for (i, row) in df.iterrows():
                if 'Sample described as:' in ' '.join(row):
                    supply_component_name = row.iloc[2]
                if 'Sampled Date & Time:' in ' '.join(row):
                    sample_date = row.iloc[-1]
                elif 'Sampled on:' in ' '.join(row):
                    sample_date = row.iloc[-1]
                if re.match('[<>\-\.\d]+',str(row.iloc[3])) is not None:
                    parameter_determinand = row.iloc[1]
                    original_value = row.iloc[3]
            
                if original_value != '':
                    output = output.append({'sample_date':sample_date,'supply_component_name':supply_component_name,'parameter_determinand':parameter_determinand,'original_value':original_value,'laboratory':laboratory,'report_number':''}, ignore_index = True)
            
            output.drop_duplicates(inplace = True)
        
        elif variant == 'vertical':
            parameter_determinand = []
            supply_component_name = []
            for (i, row) in df.iterrows():
                if 'Sample Identification' in ' '.join(row):
                    parameter_determinand = row.iloc[4:]
                elif re.findall('\d+',row[0]):
                    supply_component_name = row.iloc[1]
                    original_value = row.iloc[4:]
                elif 'Sampled on:' in ' '.join(row):
                    sample_date = row.iloc[-1]
            
            if original_value != '':
                for n in parameter_determinand:
                    output = output.append({'sample_date':sample_date,'supply_component_name':supply_component_name,'parameter_determinand':parameter_determinand[n],'original_value':original_value[n],'laboratory':laboratory,'report_number':''}, ignore_index = True)
            
            output.drop_duplicates(inplace = True)
            
    except Exception as e:
        output = e
    
    return output

def ecan_extract(df):
    output = pd.DataFrame(columns = ['sample_date','supply_component_name', 'parameter_determinand','original_value','laboratory','report_number'])
    sample_date = '',
    supply_component_name = ''
    parameter_determinand = ''
    original_value = ''
    laboratory = 'Environment Canterbury'
    report_number = ''
    
    try:
        for (i, row) in df.iterrows():
            
            if re.match('[<>\-\.\d]+',str(row.iloc[6])) is not None:
                if re.match('\w+',str(row.iloc[1])) is not None:
                    supply_component_name = row.iloc[2]
                    sample_date = row.iloc[3] 
                parameter_determinand = row.iloc[4]
                original_value = row.iloc[6]
            if 'report no:' in ' '.join(row):    
                    report_number = row.iloc[2]

            if original_value != '':
                output = output.append({'sample_date':sample_date,'supply_component_name':supply_component_name,'parameter_determinand':parameter_determinand,'original_value':original_value,'laboratory':laboratory,'report_number':report_number},  ignore_index = True)
        
            output.drop_duplicates(inplace = True)
            
    except Exception as e:
        output = e
        
    return output        
            
def data_extractor(file):
# try:
    df = pd.read_csv(file)
    df = df.iloc[1:,1:]
    df = df.drop_duplicates()
    df = df.dropna(how = 'all', axis = 0)
    df = df.dropna(how = 'all', axis = 1)
    df= df.fillna('')
    lab = ''
    
    #check which lab
    for i,row in df.iterrows():
        if lab == '':
            for n in row:
                if re.findall('eurofins',str(n).lower()):
                    lab = 'Eurofins'
                    if re.findall('sample identification',str(n).lower()):
                        variant = 'vertical'
                    else:
                        variant = 'horizontal'    

                    break
                elif re.findall('ecan',str(n).lower()) or re.findall('environment canterbury',str(n).lower()):
                    lab = 'Environment Canterbury'
                    break
                    # # print(n)
                    # lab = re.findall('ecan|environment canterbury|esr|hill lab|watercare|eurofins|hill laboratories|aqualabs|water testing|analytica laboratories ltd|analytical research lab ltd|aqualab|asurequality|broadspectrum|cawthron|central environmental labs|christchurch cc|citilab|crl energy|dargaville hospital|esr|eurofins|far north envirolab|fonterra|food & health environmental|goodman fielder, meadow fresh|hamilton cc|hill labs|industrial chemistry services|invercargill cc|clifton|linnaeus laboratory|mb century|medlab timaru|microaquatech|new zealand aluminium smelters|niwa|rotorua lc environmental lab.|sgs nz, minerals div. westport|silver fern farms - takapau|tatua co-operative dairy co|tauranga cc|waimakariri dc|wairoa dc|water testing|watercare|whangarei dc',n.lower())
                    # if lab:
                    #     print(lab)
                    #     break
    print(lab)
            
    if lab == 'Eurofins':
        data = euro_extract(df, variant)
        if type(data) == pd.DataFrame:
            data['file_name'] = [file]*data.shape[0]
    elif lab == 'Environment Canterbury':
        data = ecan_extract(df)
        if type(data) == pd.DataFrame:
            data['file_name'] = [file]*data.shape[0]
    else:
        data = 'invalid lab'
    
    return data

# except Exception as e:
#     print(e)
        
    

In [1]:
import tensorflow as tf
tf.__version__

'2.11.0'